# Iterative Simplification

The aim of this notebook is to evaluate the effectiveness of our simplification method by applying it iteratively.
> How does the simplicity of a sentence evolve when it is iteratively simplified?
> How does the similarity with the original sentence evolve?

In [1]:
# ---------------------------- PREPARING NOTEBOOK ---------------------------- #
# Autoreload
%load_ext autoreload
%autoreload 2

# Random seed
import numpy as np
np.random.seed(42)

# External modules
import os
from IPython.display import display

# Set global log level
import logging
logging.basicConfig(level=logging.INFO)
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Define PWD as the current git repository
import git
repo = git.Repo('.', search_parent_directories=True)
pwd = repo.working_dir
os.chdir(pwd)

# import

In [2]:
# -------------------------- LOAD PREVIOUS NOTEBOOKS ------------------------- #
import json
import __main__
import black
import types

paths = [
    os.path.join(pwd, "notebooks", "text_simplification", "a_DatasetCreation.ipynb"),
    os.path.join(pwd, "notebooks", "text_simplification", "c_MistralEvaluation.ipynb"),
    os.path.join(pwd, "notebooks", "text_simplification", "d_Metrics.ipynb"),
]

# Read notebooks
code_dict = {}
for path in paths:
    code = ""
    with open(path, "r") as f:
        temp = json.load(f)

    cells = [
        cell
        for cell in temp["cells"]
        if cell["cell_type"] == "code"
        and len(cell["source"]) > 0
        and cell["source"][-1] == "# import"
    ]
    notebook_code = "\n".join(
        line
        for cell in cells
        for line in cell["source"]
        if line != "# import" and len(line) > 0 and line[0] != "%"
    )
    # Create something like a header
    code += f"# {'-'*76} #\n"
    code += f"# {os.path.basename(path).upper():^76} #\n"
    code += f"# {'-'*76} #\n"
    code += notebook_code

    # Add "Module Creation"
    notebook_name = (
        os.path.basename(path).replace("imported_", "").replace(".ipynb", "")
    )
    code += """
# --------------------------------- IMPORTER --------------------------------- #
import types


class MyNotebook:
    pass


NOTEBOOK_NAME = MyNotebook()
# Put every function defined in the notebook in the class
NOTEBOOK_NAME.__dict__.update(
    {
        name: obj
        for name, obj in locals().items()
        if isinstance(obj, (type, types.FunctionType))
        if not (name.startswith("_") or name == "MyNotebook")
    }
)
    """.replace(
        "NOTEBOOK_NAME", notebook_name
    )

    # Remove empty lines
    code = "\n".join([line for line in code.split("\n") if len(line) > 0])
    # Format code
    code = black.format_str(code, mode=black.FileMode())

    # Write scrach file
    path = os.path.join(
        pwd, "scratch", f"imported_{os.path.basename(path).replace('ipynb', 'py')}"
    )
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, "w") as f:
        f.write(code)
    code_dict[path] = code


# Mainify code
for path, code in code_dict.items():
    compiled = compile(code, path, "exec")
    exec(compiled, __main__.__dict__)

# import

## Creation of the data set

In order to evaluate the evolution of the complexity and similarity of a simplified sentence, we will create a dataset of C2 sentences correctly classified by **BERT**. We will then iteratively simplify these sentences and evaluate the evolution of complexity and similarity.

In [3]:
# ----------------------------- DATASET CREATION ----------------------------- #
import pandas as pd

# Loading full dataset
test_df = c_MistralEvaluation.download_difficulty_estimation(pwd)
test_df = c_MistralEvaluation.get_balanced_dataframe(test_df, None)
test_df = test_df[test_df["Difficulty"] == "C2"]

# Get Bert predictions
predictions = d_Metrics.get_bert_difficulty_prediction(
    test_df["Sentence"], dataset="french_difficulty", pwd=os.path.join(pwd, "scratch")
)

# Keep only the sentences correctly predicted
test_df = test_df[test_df["Difficulty"] == predictions].reset_index(drop=True)

# Display
display(test_df.head())

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

/home/lopilo/.cache/pypoetry/virtualenvs/lingorank-llm-wa__1Cox-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,Sentence,Difficulty
0,Le seul point de vue raisonnable que puisse of...,C2
1,"Or, sous ce point de vue, chacune d'elles corr...",C2
2,"Préparé, au contraire, par une éducation mathé...",C2
3,"Aussi, à toutes les époques du développement s...",C2
4,"En vertu du nombre, déjà très considérable, et...",C2


## A simulation stage + Evaluation

We are now going to define a function that will perform a simplification step on a set of sentences using our **Mistral-7B** fine-tuned model. We will then evaluate the similarity and the difference in difficulty evaluated by **BERT** between the simplified sentences and the original sentences.

### Simplification step

Let's start by defining a function that takes a list of sentences as input and returns a list of simplified sentences using our **Mistral-7B** fine-tuned model.

In [4]:
# -------------------------- MISTRAL SIMPLIFICATION -------------------------- #
import os

import pandas as pd
import torch
from peft import PeftModel
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM
from tqdm import tqdm as console_tqdm
from slurmray.RayLauncher import RayLauncher
import ray

MODEL = "bofenghuang/vigostral-7b-chat"


# Define cluster function
@ray.remote(num_gpus=1)  # Indique que chaque tâche nécessite un GPU
def worker_mistral_simplify(
    test_df: pd.DataFrame,
    pwd: str = "/scratch/hjamet",
):
    # Fix partial import bug
    import ray.train.huggingface
    import ray.train.huggingface.transformers

    # Charger tokenizer
    tokenizer = a_DatasetCreation.download_tokenizer(training=False)

    # Create dataset
    dataset = a_DatasetCreation.format_data(test_df, tokenizer, training=False)

    # Encode dataset
    encoded_dataset = a_DatasetCreation.encode_dataset(dataset, tokenizer)

    # Load model
    path = os.path.join(
        pwd,
        "models",
        "difficulty_estimation",
        MODEL.replace("/", "_"),
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        os.path.join(path, "mistral_simplification_trained"),
        device_map="auto",
        use_cache=False,
        trust_remote_code=True,
    )
    model = PeftModel.from_pretrained(
        base_model, os.path.join(path, "mistral_simplification_trained")
    )

    # Move everything to GPU
    model.to("cuda")
    test_loader = DataLoader(encoded_dataset, batch_size=8)

    # Generate predictions
    with torch.no_grad():
        model.eval()
        predictions_ids = []

        for batch in console_tqdm(test_loader):
            input_ids_batch = batch["input_ids"].to("cuda")
            attention_mask_batch = batch["attention_mask"].to("cuda")

            outputs = model.generate(
                input_ids=input_ids_batch,
                attention_mask=attention_mask_batch,
                max_length=max(128, input_ids_batch.shape[1] * 2),
                num_return_sequences=1,
            )

            predictions_ids.extend(outputs)
        predictions = [
            tokenizer.decode(prediction, skip_special_tokens=True)
            for prediction in predictions_ids
        ]
        predictions_series = pd.Series(predictions)

    return predictions_series


def cluster_mistral_simplify(test_df: pd.DataFrame, pwd: str = "/scratch/hjamet"):
    # Split the DataFrame into chunks for parallel processing
    num_gpus = ray.cluster_resources()["GPU"]
    chunk_size = int(len(test_df) / num_gpus)
    chunks = [test_df[i : i + chunk_size] for i in range(0, len(test_df), chunk_size)]

    # Distribute the work to the GPUs
    futures = [worker_mistral_simplify.remote(chunk, pwd) for chunk in chunks]

    # Collect the results from the GPUs
    predictions_series = pd.concat(ray.get(futures))

    return predictions_series


# Define launcher function
def mistral_simplify(test_df: pd.DataFrame, save_n: int = None, ssh_key: str = None):
    # Define ray launcher
    launcher = RayLauncher(
        project_name="mistral_sentence_simplification",
        func=cluster_mistral_simplify,
        args={"test_df": test_df, "pwd": "/scratch/hjamet"},
        modules=[],
        node_nbr=3,
        use_gpu=True,
        memory=128,
        max_running_time=60,
        server_run=True,
        server_ssh="curnagl.dcsr.unil.ch",
        server_username="hjamet",
        server_password=ssh_key,
    )

    # Run inference
    predictions = launcher()

    # Save predictions
    if save_n is not None:
        path = os.path.join(
            pwd,
            "results",
            "text_simplification",
            "IterativeSimplification",
        )
        if not os.path.exists(path):
            os.makedirs(path)
        predictions.to_csv(
            os.path.join(
                path,
                f"{save_n}_simplified_raw.csv",
            ),
            index=False,
        )

    # Format predictions
    predictions_df = pd.concat(
        [
            test_df.reset_index(drop=True),
            predictions.str.extract(r"\[/INST\] (.*[\.\n])")
            .iloc[:, 0]
            .rename("Simplified")
            .str.strip()
            .reset_index(drop=True),
        ],
        axis=1,
    )

    # Save formatted predictions
    if save_n is not None:
        predictions_df.to_csv(
            os.path.join(
                path,
                f"{save_n}_simplified_formatted.csv",
            ),
            index=False,
        )

    return predictions_df

In [6]:
predictions_df = mistral_simplify(test_df.iloc[:10])

Serializing function and arguments...
Connecting to the cluster...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


Writing slurmray server script...
Running server...
Installing slurmray server
Writing python script...
Writing slurm script...
No serialization done.
Cluster detected, running on cluster...
Canceling old jobs...
Start to submit job!
Job submitted! Script file is at: </users/hjamet/slurmray-server/.slogs/server/sbatch.sh>. Log file is at: </users/hjamet/slurmray-server/.slogs/server/server_1402-16h08.log>
Start to monitor the queue... You can check the queue at: </users/hjamet/slurmray-server/.slogs/server/server_1402-16h08_queue.log>
Submitted batch job 38690509
IP Head: 10.203.101.82:6379
STARTING HEAD at dnagpu002
2024-02-14 16:08:28,020	INFO usage_lib.py:449 -- Usage stats collection is enabled by default without user confirmation because this terminal is detected to be non-interactive. To disable this, add `--disable-usage-stats` to the command that starts the cluster, or run the following command: `ray disable-usage-stats` before starting the cluster. See https://docs.ray.io/en/m

### Assessment

We are now going to evaluate the characteristics of the simplified sentences compared to the original sentences. To do this, we will use **BERT** to evaluate :
- The similarity between the simplified sentences and the original sentences
- The level of difficulty of the simplified sentences.

In [7]:
# ------------------------- BERT EVALUATION FUNCTION ------------------------- #
def bert_simplification_evaluation(
    predictions_df: pd.DataFrame,
    save_n: int = None,
):
    # Similarity
    similarity = d_Metrics.get_simplification_similarity(
        predictions_df.rename(columns={"Sentence": "Original"}),
    ).similarity
    predictions_df["Similarity"] = similarity

    # Estimated Difficulty
    difficulty = d_Metrics.get_bert_difficulty_prediction(
        predictions_df["Simplified"],
        dataset="french_difficulty",
        pwd=os.path.join(pwd, "scratch"),
    )
    predictions_df["Estimated_Difficulty"] = difficulty

    # Save predictions
    if save_n is not None:
        path = os.path.join(
            pwd,
            "results",
            "text_simplification",
            "IterativeSimplification",
        )
        if not os.path.exists(path):
            os.makedirs(path)
        predictions_df.to_csv(
            os.path.join(
                path,
                f"{save_n}_simplified_evaluation.csv",
            ),
            index=False,
        )

    return predictions_df

In [8]:
bert_simplification_evaluation(predictions_df)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/home/lopilo/.cache/pypoetry/virtualenvs/lingorank-llm-wa__1Cox-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,Sentence,Difficulty,Simplified,Similarity,Estimated_Difficulty
0,Le seul point de vue raisonnable que puisse of...,C2,La seule façon raisonnable de voir un principe...,0.942184,C1
1,"Or, sous ce point de vue, chacune d'elles corr...",C2,"Donc, selon cette approche, chaque lune corres...",0.966188,C2
2,"Préparé, au contraire, par une éducation mathé...",C2,"Bien éduqué en mathématiques et en astronomie,...",0.854498,C1
3,"Aussi, à toutes les époques du développement s...",C2,Depuis que la chimie est devenue une science à...,0.921910,B1
4,"En vertu du nombre, déjà très considérable, et...",C2,Beaucoup de corps chimiques simples sont étudi...,0.928633,C1
5,"Mais, à partir des événemens physiques, la sus...",C2,Mais si on regarde les choses qui se passent d...,0.889296,B1
6,Malheureusement l'admirable extension de la pu...,C2,"Malheureusement, l'extension de la chimie aujo...",0.894255,B2
7,"Car, d'après les considérations sommairement i...",C2,"Donc, selon ce que l'on a vu dans la leçon pré...",0.918746,B1
8,"Sous ce premier rapport, qui est décisif, la p...",C2,"Dans ce rapport important, on décide de la pla...",0.933345,B2
9,"Qui pourrait méconnaître aujourd'hui que, par ...",C2,Il est difficile de ne pas reconnaître aujourd...,0.892081,C2


### Iterative simplification Step

Finally, we just need to define a function that performs the simplification and evaluation iteratively.

In [10]:
from tqdm import notebook as notebook_tqdm
from getpass import getpass


def iterative_simplification(test_df: pd.DataFrame, n: int = 8, bert : bool = False):
    # Store Originals
    result = (
        test_df.copy().rename(columns={"Sentence": "Original"}).reset_index(drop=True)
    )

    # Ask for password
    ssh_key = getpass("Enter your cluster password: ")

    temp_df = test_df.copy()
    for i in notebook_tqdm.tqdm(range(n)):
        # Simplify
        temp_df = mistral_simplify(temp_df, save_n=i + 1, ssh_key=ssh_key)

        # Evaluate
        temp_df["Sentence"] = result["Original"]
        temp_df = temp_df.fillna("")
        temp_df = bert_simplification_evaluation(temp_df, save_n=i + 1)

        # Store
        temp_to_store = temp_df[["Simplified", "Similarity", "Estimated_Difficulty"]]
        temp_to_store.columns = [
            f"Simplified_{i}",
            f"Similarity_{i}",
            f"Difficulty_{i}",
        ]
        result = pd.concat([result, temp_to_store], axis=1)

        # For safety, save the result
        path = os.path.join(
            pwd,
            "scratch",
            "IterativeSimplification",
        )
        if not os.path.exists(path):
            os.makedirs(path)
        result.to_csv(
            os.path.join(
                path,
                "iterative_simplification.csv",
            ),
            index=False,
        )

        # Prepare next iteration
        temp_df = temp_df[["Simplified", "Estimated_Difficulty"]]
        temp_df.columns = ["Sentence", "Difficulty"]
        
        # Change Difficulty to real difficulty
        if not(bert):
            temp_df["Difficulty"] = ["C1", "B2", "B1", "A2"][min(i, 3)]

    return result

## Final Evaluation

Finally, we're going to produce our results using the entire dataset.

In [12]:
# Run iterative simplification
result = iterative_simplification(test_df.sample(100))

# Save result
result.dropna().to_csv(
    os.path.join(
        pwd,
        "results",
        "text_simplification",
        "IterativeSimplification",
        "iterative_simplification_label.csv",
    ),
    index=False,
)

  0%|          | 0/8 [00:00<?, ?it/s]

Serializing function and arguments...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)


Connecting to the cluster...


INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


...

Result downloaded!


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

/home/lopilo/.cache/pypoetry/virtualenvs/lingorank-llm-wa__1Cox-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
# Run iterative simplification
result = iterative_simplification(test_df.sample(100), bert=True)

# Save result
result.dropna().to_csv(
    os.path.join(
        pwd,
        "results",
        "text_simplification",
        "IterativeSimplification",
        "iterative_simplification_bert.csv",
    ),
    index=False,
)

  0%|          | 0/8 [00:00<?, ?it/s]

Serializing function and arguments...


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.0)


Connecting to the cluster...


INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)


...

Result downloaded!


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

/home/lopilo/.cache/pypoetry/virtualenvs/lingorank-llm-wa__1Cox-py3.9/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
